# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Csv and DataFrame used for plotting in WeatherPy assignment

openweather = pd.read_csv("Resources/openweather.csv")
openweather.head()

,City,Country,Temperature,Humidity %,Cloudiness %,Wind Speed (mph),Latitude,Longitude,Date
0,sitka,US,55.40,62,1,4.41,57.05,-135.33,1600107974
1,tiksi,RU,40.24,87,60,9.33,71.69,128.87,1600107854
2,kununurra,AU,80.60,24,40,10.29,-15.77,128.73,1600108074
3,ushuaia,AR,42.80,59,20,4.70,-54.80,-68.30,1600108074
4,buin,CL,80.60,12,0,2.24,-33.73,-70.75,1600108075


In [3]:
openweather.dtypes

City                 object
Country              object
Temperature         float64
Humidity %            int64
Cloudiness %          int64
Wind Speed (mph)    float64
Latitude            float64
Longitude           float64
Date                  int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store Latitude and Longitude in Locations
locations = openweather[["Latitude", "Longitude"]]

# Convert Humidity astype float
humidity = openweather["Humidity %"].astype(float)

# Plot Heatmap
figure_layout = {'width': '1000px',
                'height': '500px',
                'border': '1px solid black',
                'padding': 'px',
                'margin': '0 auto 0 auto'
                }
fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(15,25))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=1.5)

# Add Layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='px', wid…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Criteria and Ideal temperature
criteria_temp = (openweather['Temperature'] > 70) & (openweather['Temperature'] < 80)
criteria_windspeed = openweather["Wind Speed (mph)"] < 15
criteria_cloudiness = openweather["Cloudiness %"] == 0
criteria = criteria_temp & criteria_windspeed & criteria_cloudiness

In [6]:
# Ideal Dataframe based on Criteria
ideal_weather = openweather[criteria]
ideal_weather.head()

,City,Country,Temperature,Humidity %,Cloudiness %,Wind Speed (mph),Latitude,Longitude,Date
30,rikitea,PF,73.56,80,0,11.77,-23.12,-134.97,1600107963
236,donskoye,RU,71.60,35,0,4.47,45.46,41.97,1600108107
340,newport,GB,75.99,71,0,4.14,51.59,-3.00,1600107944
347,melipilla,CL,78.80,54,0,6.93,-33.70,-71.22,1600108127
384,abu samrah,SY,77.36,41,0,8.10,35.30,37.18,1600108132


In [7]:
ideal_weather.to_csv("Output/idealweather.csv", index=False)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# New Dataframe for Hotel
# Add Column for Hotel Name
hotel_df = ideal_weather
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.reset_index()
hotel_df.head()

C:\Users\crism\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,index,City,Country,Temperature,Humidity %,Cloudiness %,Wind Speed (mph),Latitude,Longitude,Date,Hotel Name
0,30,rikitea,PF,73.56,80,0,11.77,-23.12,-134.97,1600107963,
1,236,donskoye,RU,71.60,35,0,4.47,45.46,41.97,1600108107,
2,340,newport,GB,75.99,71,0,4.14,51.59,-3.00,1600107944,
3,347,melipilla,CL,78.80,54,0,6.93,-33.70,-71.22,1600108127,
4,384,abu samrah,SY,77.36,41,0,8.10,35.30,37.18,1600108132,


In [9]:
# Set parameters to search for hotels with 5000 meters.
params = {"radius": 5000,
          "type": "lodging",
          "keyword": "Hotel",
          "key": g_key         
         }

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    hotel = response.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result...skipping")  

Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping


In [10]:
hotel_df.shape

(9, 11)

In [11]:
hotel_df.head(10)

,index,City,Country,Temperature,Humidity %,Cloudiness %,Wind Speed (mph),Latitude,Longitude,Date,Hotel Name
0,30,rikitea,PF,73.56,80,0,11.77,-23.12,-134.97,1600107963,Pension Bianca & Benoit
1,236,donskoye,RU,71.60,35,0,4.47,45.46,41.97,1600108107,
2,340,newport,GB,75.99,71,0,4.14,51.59,-3.00,1600107944,The Celtic Manor Resort
3,347,melipilla,CL,78.80,54,0,6.93,-33.70,-71.22,1600108127,Hotel Los Troncos
4,384,abu samrah,SY,77.36,41,0,8.10,35.30,37.18,1600108132,
5,497,maputo,MZ,75.00,83,0,13.87,-25.97,32.59,1600107903,Southern Sun Maputo
6,521,veere,NL,78.80,47,0,6.93,51.55,3.67,1600108154,Fletcher Hotel-Restaurant Het Veerse Meer
7,537,sartana,UA,73.00,100,0,7.00,47.17,37.70,1600108157,
8,542,port keats,AU,75.38,85,0,8.41,-14.25,129.55,1600108157,TDC Village (Hundred Man Camp)


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
figure_layout = {'width': '1000px',
                 'height': '500px',
                 'border': '1px solid black',
                 'padding': '1px',
                 'margin': '0 auto 0 auto'
                }

fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(15,25))

# Create Hotel picks
hotel_picks = gmaps.marker_layer(locations, info_box_content = [info_box_template.format(**row) for index,
                                                               row in hotel_df.iterrows()])

# Display figure
fig.add_layer(heat_layer)
fig.add_layer(hotel_picks)
fig


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…